In [9]:
import os
import groq
from dotenv import load_dotenv
from groq import Groq

load_dotenv()

GROQ_API_KEY = os.environ['GROQ_API_KEY']

model = "llama-3.3-70b-versatile"

client = Groq(api_key=GROQ_API_KEY)

### Formatting model response as JSON

In [ ]:
# You need to specify the model that you want a json response
prompt = """
I have these notes with book titles and authors: New releases this week! The Beholders by Hester Musson, 
The Mystery Guest by Nita Prose. Please organize the titles and authors in a json file.
"""

response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt}],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

{
   "books": [
       {
           "title": "The Beholders",
           "author": "Hester Musson"
       },
       {
           "title": "The Mystery Guest",
           "author": "Nita Prose"
       }
   ]
}


### Handling Errors

In [ ]:
# - NotFoundError: Model not found
try:
    response = client.chat.completions.create(
        model="this is a model that does not exist",
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )
except groq.NotFoundError as e:
    print(e)

Error code: 404 - {'error': {'message': 'The model `this is a model that does not exist` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'code': 'model_not_found'}}


In [ ]:
# Connection errors:
# - InternalServerError
# - APIConectionError
# - APITimeoutError
# Potential solution: Checking connection config and reaching out to Groq support

# - AuthenticationError: Invalid API key
client = Groq(api_key="this is a fake API key")
try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format={"type": "json_object"}
    )
except groq.InternalServerError as e:
    print(e)
except groq.APIConnectionError as e:
    print(e)
except groq.APITimeoutError as e:
    print(e)
except groq.AuthenticationError as e:
    print(e)

Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}


In [ ]:
# Resource limit errors:
# - ConflictError
# - RateLimitError
# Solutions: Checking limit restrictions and ensure requests are within limits

In [19]:
# Bad requests errors
# - BadRequestError

client = groq.Groq(api_key=GROQ_API_KEY)
try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "not valid role", "content": prompt}],
        response_format={"type": "json_object"}
    )
except groq.BadRequestError as e:
    print(e)

Error code: 400 - {'error': {'message': "'messages.0' : discriminator property 'role' has invalid value", 'type': 'invalid_request_error'}}


In [22]:
# Handling the exceptions

try:
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": "List five data science professions"}]
    )
except groq.AuthenticationError as e:
    print(f"Groq API failed to authenticate: {e}")
except groq.NotFoundError as e:
    print(f"Model not found: {e}")
except groq.RateLimitError as e:
    print(f"Groq API rate limit exceeded: {e}")
except Exception as e:
    print(f"Unable to generate response. Error: {e}")
finally:
    print(response.choices[0].message.content)

Here are five data science professions:

1. **Data Scientist**: A data scientist is responsible for collecting, analyzing, and interpreting complex data to gain insights and inform business decisions. They use machine learning, statistical models, and data visualization to solve problems and drive business outcomes.

2. **Data Analyst**: A data analyst works with data to identify trends, create reports, and develop visualizations to help organizations make informed decisions. They use tools like Excel, SQL, and Tableau to analyze and present data insights.

3. **Machine Learning Engineer**: A machine learning engineer designs and develops artificial intelligence and machine learning models to solve complex problems. They work on building, testing, and deploying models that can learn from data and make predictions or recommendations.

4. **Business Intelligence Developer**: A business intelligence developer creates data visualizations, reports, and dashboards to help organizations under

### Retrying

In [ ]:
# You can retry if the request fails with tenacity package
# - stop_after_attempt: stops after a certain number of attempts
# - wait_random_exponential: waits for a random amount of time between attempts
from tenacity import retry, stop_after_attempt, wait_random_exponential

@retry(stop=stop_after_attempt(3), wait=wait_random_exponential(min=5, max=10))
def get_response(model, message):
    response = client.chat.completions.create(
        model=model,
        messages=[message]
    )
    return response.choices[0].message.content

get_response(model, {"role": "user", "content": "List five data science professions"})

'Here are five data science professions:\n\n1. **Data Scientist**: A data scientist collects, analyzes, and interprets complex data to gain insights and inform business decisions. They use statistical models, machine learning algorithms, and data visualization techniques to extract knowledge from data.\n\n2. **Data Analyst**: A data analyst works with data to identify trends, create reports, and develop visualizations to help organizations make informed decisions. They use tools like Excel, SQL, and Tableau to analyze and present data.\n\n3. **Machine Learning Engineer**: A machine learning engineer designs and develops artificial intelligence and machine learning systems that can learn from data and make predictions or take actions. They work on building predictive models, natural language processing, and computer vision.\n\n4. **Business Intelligence Developer**: A business intelligence developer creates data visualizations, reports, and dashboards to help organizations understand th

### Batching

In [30]:
# Batching referrers to sending multiple requests in a single call
# It is useful when you have multiple requests to make and you want to optimize the number of API calls

def get_response(messages):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

measurements = [5.2, 6.3, 3.7]
messages = []
messages.append({
    "role": "system", 
    "content": "Your task is to convert each measure in kilometers to miles. Your output should be a table with all the measurements."
})
[messages.append({"role": "user", "content": str(i)}) for i in measurements]

response = get_response(messages)
print(response)

To convert kilometers to miles, we multiply by 0.621371. Here are the conversions:

| Kilometers | Miles |
| --- | --- |
| 5.2 | 3.23 |
| 6.3 | 3.91 |
| 3.7 | 2.30 |


### Setting token limits

In [35]:
# You can set token limits to control the number of requests made
# For that you can use tiktoken package
# tiktoken solo funciona para modelos de OpenAI
# import tiktoken

# input_message = {
#     "role": "user", 
#     "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"
# }

# encoding = tiktoken.encoding_for_model(model)
# num_tokens = len(encoding.encode(input_message["content"]))

In [ ]:
# Al parecer para usar modelos llama en huggingface
# se necesita pedir permiso dentro de HF en https://huggingface.co/meta-llama/Llama-3.3-70b-Instruct
# Luego de obtener el permiso se tiene que:
# - Crear un access token de HF en settings (https://huggingface.co/settings/tokens)
# - Correr el siguiente comando: huggingface-cli login
# - Copiar y pegar el token y listo.

In [ ]:
# En vez de tiktoken, podemos usar transformers de huggingface
from transformers import AutoTokenizer

input_message = {
    "role": "user", 
    "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"
}

model_id = "meta-llama/Llama-3.3-70b-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokens = tokenizer.encode(input_message["content"], return_tensors="pt")

num_tokens = len(tokens[0])

print("num_tokens:", num_tokens)

if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.3-70b-Instruct.
403 Client Error. (Request ID: Root=1-67ca7987-6422d35e59e361947dcb7c22;9382fd77-07ad-40bd-87db-f019e54326df)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.3-70b-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.3-70B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Llama-3.3-70B-Instruct to ask for access.